In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

train_dir = '../데이터셋/한국인 감정인식을 위한 복합 영상/Training/split_data/train'
val_dir = '../데이터셋/한국인 감정인식을 위한 복합 영상/Training/split_data/val'
test_dir = '../데이터셋/한국인 감정인식을 위한 복합 영상/Training/split_data/test'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')



Found 78413 images belonging to 7 classes.
Found 16803 images belonging to 7 classes.
Found 16803 images belonging to 7 classes.


In [7]:
print(tf.__version__)

2.10.0


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15456538177108028626
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5758779392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4119326416522053438
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [9]:
tf.test.is_built_with_cuda()

True

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [7]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size)

Epoch 1/25
2450/2450 [==============================] - 275s 112ms/step - loss: 1.2487 - accuracy: 0.5210 - val_loss: 1.2734 - val_accuracy: 0.5017
Epoch 2/25
2450/2450 [==============================] - 268s 109ms/step - loss: 1.1237 - accuracy: 0.5731 - val_loss: 1.2767 - val_accuracy: 0.5096
Epoch 3/25
2450/2450 [==============================] - 271s 110ms/step - loss: 0.9862 - accuracy: 0.6311 - val_loss: 1.3111 - val_accuracy: 0.5065
Epoch 4/25
2450/2450 [==============================] - 272s 111ms/step - loss: 0.8509 - accuracy: 0.6858 - val_loss: 1.4014 - val_accuracy: 0.5115
Epoch 5/25
2450/2450 [==============================] - 246s 100ms/step - loss: 0.7308 - accuracy: 0.7312 - val_loss: 1.4294 - val_accuracy: 0.5118
Epoch 6/25
2450/2450 [==============================] - 222s 91ms/step - loss: 0.6404 - accuracy: 0.7653 - val_loss: 1.6229 - val_accuracy: 0.5076
Epoch 7/25
2450/2450 [==============================] - 179s 73ms/step - loss: 0.5533 - accuracy: 0.7989 - val_lo

In [13]:
model.evaluate(test_generator)


526/526 [==============================] - 39s 73ms/step - loss: 3.1729 - accuracy: 0.5051


[3.1729114055633545, 0.5051478743553162]